In [2]:
%cd /kaggle/working/ViXTTS


/kaggle/working/ViXTTS


In [3]:
!pip install trainer==0.0.36
!pip install transformers==4.44.2
!pip install spacy==3.7.5
!pip install numpy==1.23.5
!pip install mutagen


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.0
    Uninstalling tokenizers-0.20.0:
      Successfully uninstalled tokenizers-0.20.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 48.1 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.6
    Uninstalling spacy-3.7.6:
      Successfully uninstalled spacy-3.7.6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 77.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    

In [4]:
!pip install TTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 41.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 85.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 51.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 99.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 76.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
INFO: pip is looking at multiple versions of librosa to determine which vers

In [5]:
import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager

In [6]:
# Logging parameters
RUN_NAME = "GPT_XTTS_v2.0_vietnamese_FT"
PROJECT_NAME = "XTTS_trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

In [7]:
import os

# Đường dẫn tới folder mới
OUT_PATH  = os.path.join(os.getcwd(), 'output')

# Tạo folder mới nếu nó chưa tồn tại
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)

print(f'Folder "output" đã được tạo tại: {OUT_PATH}')


Folder "output" đã được tạo tại: /kaggle/working/ViXTTS/output


In [8]:
# Training Parameters
OPTIMIZER_WD_ONLY_ON_WEIGHTS = True  # for multi-gpu training please make it False
START_WITH_EVAL = True  # if True it will star with evaluation
BATCH_SIZE = 3  # set here the batch size
GRAD_ACUMM_STEPS = 84  # set here the grad accumulation steps
# Note: we recommend that BATCH_SIZE * GRAD_ACUMM_STEPS need to be at least 252 for more efficient training. You can increase/decrease BATCH_SIZE but then set GRAD_ACUMM_STEPS accordingly.


In [9]:
# Define here the dataset that you want to use for the fine-tuning on.
config_dataset = BaseDatasetConfig(
    formatter="coqui",  # Tên hàm định dạng bạn đã định nghĩa
    dataset_name="ViXTTS_dataset",  # Tên dataset
    path=os.path.join(OUT_PATH, "/kaggle/input/vixtts-dataset"),  # Đường dẫn đến thư mục chứa dataset
    meta_file_train=os.path.join(OUT_PATH, "/kaggle/input/vixtts-dataset/metadata.csv"),  # Đường dẫn đến file metadata train)
    language='vi'
)


In [10]:
# Add here the configs of the datasets
DATASETS_CONFIG_LIST = [config_dataset]

In [11]:
import shutil
import os

# Định nghĩa đường dẫn của các file nguồn
source_files = [
    "/kaggle/input/finetuning-vixtts-gpt2/best_model.pth",
    "/kaggle/input/finetuning-vixtts-gpt2/config.json",
    "/kaggle/input/finetuning-vixtts-gpt2/vocab.json"
]

# Đường dẫn đến thư mục output
destination_folder = "/kaggle/working/"

# Sao chép từng file từ source_files đến destination_folder
for file in source_files:
    # Nếu file là best_model.pth, đổi tên khi sao chép
    if os.path.basename(file) == "best_model.pth":
        shutil.copy(file, os.path.join(destination_folder, "old_best_model.pth"))
    else:
        shutil.copy(file, destination_folder)

print("Các file đã được sao chép thành công!")


Các file đã được sao chép thành công!


In [12]:
# Define the path where XTTS v2.0.1 files will be downloaded
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)


# DVAE files
DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

# Set the path to the downloaded files
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

# download DVAE files if needed
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)


# # Download XTTS v2.0 checkpoint if needed
# TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
# XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"
# XTTS_CONFIG_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/config.json"

# XTTS transfer learning parameters: You we need to provide the paths of XTTS model checkpoint that you want to do the fine tuning.
TOKENIZER_FILE = "/kaggle/working/vocab.json"  # vocab.json file
XTTS_CHECKPOINT = "/kaggle/working/old_best_model.pth"  # model.pth file
XTTS_CONFIG_FILE = "/kaggle/working/config.json"

# download XTTS v2.0 files if needed
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CONFIG_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True
    )

 > Downloading DVAE files!


  0%|          | 0.00/1.07k [00:00<?, ?iB/s]
100%|██████████| 1.07k/1.07k [00:00<00:00, 2.45kiB/s]

  1%|          | 1.11M/211M [00:00<00:19, 10.8MiB/s]
  1%|          | 2.19M/211M [00:00<00:19, 10.5MiB/s]
  2%|▏         | 3.25M/211M [00:00<00:19, 10.5MiB/s]
  2%|▏         | 4.30M/211M [00:00<00:19, 10.4MiB/s]
  3%|▎         | 5.34M/211M [00:00<00:19, 10.4MiB/s]
  3%|▎         | 6.42M/211M [00:00<00:19, 10.5MiB/s]
  4%|▎         | 7.48M/211M [00:00<00:19, 10.5MiB/s]
  4%|▍         | 8.52M/211M [00:00<00:19, 10.4MiB/s]
  5%|▍         | 9.57M/211M [00:00<00:19, 10.4MiB/s]
  5%|▌         | 10.6M/211M [00:01<00:19, 10.4MiB/s]
  6%|▌         | 11.7M/211M [00:01<00:19, 10.4MiB/s]
  6%|▌         | 12.7M/211M [00:01<00:18, 10.4MiB/s]
  7%|▋         | 13.7M/211M [00:01<00:18, 10.4MiB/s]
  7%|▋         | 14.8M/211M [00:01<00:18, 10.4MiB/s]
  8%|▊         | 15.8M/211M [00:01<00:18, 10.4MiB/s]
  8%|▊         | 16.9M/211M [00:01<00:18, 10.4MiB/s]
  9%|▊         | 17.9M/211M [00:01<00:18, 10.5MiB/s]

In [13]:
def main():
    # init args and config
    model_args = GPTArgs(
        max_conditioning_length=132300,  # 6 secs
        min_conditioning_length=66150,  # 3 secs
        debug_loading_failures=False,
        max_wav_length=255995,  # ~11.6 seconds
        max_text_length=200,
        mel_norm_file=MEL_NORM_FILE,
        dvae_checkpoint=DVAE_CHECKPOINT,
        xtts_checkpoint=XTTS_CHECKPOINT,  # checkpoint path of the model that you want to fine-tune
        tokenizer_file=TOKENIZER_FILE,
        gpt_num_audio_tokens=1026,
        gpt_start_audio_token=1024,
        gpt_stop_audio_token=1025,
        gpt_use_masking_gt_prompt_approach=True,
        gpt_use_perceiver_resampler=True,
    )
    # define audio config
    audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=24000)
    # training parameters config
    config = GPTTrainerConfig(
        output_path=OUT_PATH,
        model_args=model_args,
        run_name=RUN_NAME,
        project_name=PROJECT_NAME,
        run_description="""
            GPT XTTS training
            """,
        dashboard_logger=DASHBOARD_LOGGER,
        logger_uri=LOGGER_URI,
        audio=audio_config,
        batch_size=BATCH_SIZE,
        batch_group_size=48,
        eval_batch_size=BATCH_SIZE,
        num_loader_workers=8,
        eval_split_max_size=256,
        print_step=50,
        plot_step=100,
        log_model_step=1000,
        save_step=80000,
        save_n_checkpoints=1,
        save_checkpoints=True,
        # target_loss="loss",
        print_eval=False,
        # Optimizer values like tortoise, pytorch implementation with modifications to not apply WD to non-weight parameters.
        optimizer="AdamW",
        optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
        optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
        lr=5e-06,  # learning rate
        lr_scheduler="MultiStepLR",
        # it was adjusted accordly for the new step scheme
        lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
#         test_sentences=[
#             {
#                 "text": "It took me quite a long time to develop a voice, and now that I have it I'm not going to be silent.",
#                 "speaker_wav": SPEAKER_REFERENCE,
#                 "language": LANGUAGE,
#             },
#             {
#                 "text": "This cake is great. It's so delicious and moist.",
#                 "speaker_wav": SPEAKER_REFERENCE,
#                 "language": LANGUAGE,
#             },
#         ],
    )

    # init the model from config
    model = GPTTrainer.init_from_config(config)

    # load training samples
    train_samples, eval_samples = load_tts_samples(
        DATASETS_CONFIG_LIST,
        eval_split=True,
        eval_split_max_size=config.eval_split_max_size,
        eval_split_size=config.eval_split_size,
    )

    # init the trainer and 🚀
    trainer = Trainer(
        TrainerArgs(
            restore_path=None,  # xtts checkpoint is restored via xtts_checkpoint key so no need of restore it using Trainer restore_path parameter
            skip_train_epoch=True,
            start_with_eval=START_WITH_EVAL,
            grad_accum_steps=GRAD_ACUMM_STEPS,
        ),
        config,
        output_path=OUT_PATH,
        model=model,
        train_samples=train_samples,
        eval_samples=eval_samples,
    )
    trainer.fit()

In [14]:

if __name__ == "__main__":
    main()

/kaggle/working/ViXTTS/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)
/kaggle/working/ViXTTS/TTS

>> DVAE weights restored from: /kaggle/working/ViXTTS/output/XTTS_v2.0_original_model_files/dvae.pth



100%|██████████| 211M/211M [00:39<00:00, 10.4MiB/s]

 | > [!] 5605 files not found


 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 4
 | > Num. of Torch Threads: 1
 | > Torch seed: 1
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False


...

 > Start Tensorboard: tensorboard --logdir=/kaggle/working/ViXTTS/output/GPT_XTTS_v2.0_vietnamese_FT-November-04-2024_04+11PM-9e05575

 > Model has 522064679 parameters

 > EPOCH: 0/1000
 --> /kaggle/working/ViXTTS/output/GPT_XTTS_v2.0_vietnamese_FT-November-04-2024_04+11PM-9e05575


 > Filtering invalid eval samples!!



 > EVALUATION 



 > Total eval samples after filtering: 256



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.10377156313727884 (+0)
     | > avg_loss_text_ce: 0.05103354186696165 (+0)
     | > avg_loss_mel_ce: 2.959471811967737 (+0)
     | > avg_loss: 3.0105053508982946 (+0)


 > EPOCH: 1/1000
 --> /kaggle/working/ViXTTS/output/GPT_XTTS_v2.0_vietnamese_FT-November-04-2024_04+11PM-9e05575

 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.10598169214585249 (+0.002210129008573647)
     | > avg_loss_text_ce: 0.0510335418231347 (-4.382694901439166e-11)
     | > avg_loss_mel_ce: 2.9594717979431153 (-1.4024621464159281e-08)
     | > avg_loss: 3.0105053368736723 (-1.4024622352337701e-08)


 > EPOCH: 2/1000
 --> /kaggle/working/ViXTTS/output/GPT_XTTS_v2.0_vietnamese_FT-November-04-2024_04+11PM-9e05575

 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.1099942824419807 (+0.004012590296128216)
     | > avg_loss_text_ce: 0.0510335418231347 (+0.0)
     | > avg_loss_mel_ce: 2.9594718035529644 (+5.6098490297529224e-09)
